In [1]:
import gensim
#import fasttext as ft
import re
import itertools
import collections 
import nltk
import pandas as pd
import numpy as np
import string

In [2]:
articles = pd.read_csv('Tweets.csv')
#articles = articles1.iloc[:100,:]

In [3]:
import time
start = time.time()

In [6]:
def topic_generation(x):
    doc_complete = nltk.sent_tokenize(x.decode('utf-8')) # convert each training example to a list of documents
    stop = set(nltk.corpus.stopwords.words('english')) #to remove stopwords
    exclude = set(string.punctuation)                     #to remove punctuations
    lemma = nltk.stem.wordnet.WordNetLemmatizer()       #to correct slang and return the base word
    
    def clean(doc):
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return normalized                             

    doc_clean = [clean(doc).split() for doc in doc_complete]   #list of docs where each doc contains its important words
    dictionary = gensim.corpora.Dictionary(doc_clean)          

    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    Lda = gensim.models.ldamodel.LdaModel

    # Running and Trainign LDA model on the document term matrix.
    try:
        ldamodel = Lda(doc_term_matrix, num_topics=len(doc_complete), id2word = dictionary, passes=20)
        topic_list = []

        topics = ldamodel.show_topics(num_topics = len(doc_complete), num_words = 1) # num_words-no.of words under 1 topic(the one with the highest prob. for that topic)
        for i in range(len(topics)):
            if topics[i][1][7:-1] not in topic_list:
                topic_list.append(topics[i][1][7:-1])
        return topic_list

    except ValueError:
        pass
   
articles['topics'] = articles.text.map(lambda x:topic_generation(x))

In [7]:
articles['topics']

0                                [dhepburn]
1                                   [added]
2                                 [another]
3                              [aggressive]
4                                     [bad]
5                         [playing, flying]
6                                    [away]
7         [httpstcomwpg7grezp, opportunity]
8                                 [well, d]
9                             [early, good]
10                                   [1024]
11                [d, graphic, iconography]
12                 [p, trip, virginamerica]
13                         [fabulous, take]
14                                 [thanks]
15                                    [mia]
16                      [cross, 29daystogo]
17                              [help, nyc]
18                     [virginamerica, ☺️👍]
19                 [please, fly, amazingly]
20                          [time, carrier]
21       [virginamerica, httptcout5grrwaaa]
22                             [

In [8]:
stop=time.time()

In [10]:
stop-start

954.9847421646118